In [91]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns



from sklearn.model_selection import train_test_split, KFold , cross_val_score


from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline , make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, StandardScaler


from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb




plt.style.use('fivethirtyeight')

In [7]:
X_full_train = pd.read_csv('train.csv')
X_full_test  = pd.read_csv('test.csv')

In [8]:
X_full_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
drop_column = ['PassengerId','Cabin', 'Ticket','Name']

X = X_full_train.drop(columns=['Survived'])
X.drop(drop_column, axis=1, inplace = True)

y = X_full_train['Survived']

In [10]:
drop_column = ['PassengerId','Cabin', 'Ticket','Name']
X_full_test = X_full_test.drop(columns=drop_column)

In [11]:
X_full_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S


In [12]:
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [13]:
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

## EDA

In [14]:
X.dtypes

Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked     object
dtype: object

In [15]:
X.describe()

,Pclass,Age,SibSp,Parch,Fare
count,891.000000,714.000000,891.000000,891.000000,891.000000
mean,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.420000,0.000000,0.000000,0.000000
25%,2.000000,20.125000,0.000000,0.000000,7.910400
50%,3.000000,28.000000,0.000000,0.000000,14.454200
75%,3.000000,38.000000,1.000000,0.000000,31.000000
max,3.000000,80.000000,8.000000,6.000000,512.329200


In [16]:
X.describe(exclude=np.number)

,Sex,Embarked
count,891,889
unique,2,3
top,male,S
freq,577,644


In [17]:
#Agrupando features numéricas
numeric_features  = [cls for cls in X.columns if X[cls].dtype == 'float64' or X[cls].dtype =='int64']
print('>>numeric_features:', numeric_features ,
      len(numeric_features))


#Agrupando features categoricas
cat_features = [cls for cls in X.columns if X[cls].dtypes == 'object' and X[cls].nunique()<=3]
print('\n>>cat_features:', cat_features ,
      len(cat_features))

>>numeric_features: ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'] 5

>>cat_features: ['Sex', 'Embarked'] 2


## Split

In [18]:
Xtrain,Xval,ytrain,yval = train_test_split(X,y,test_size=0.2,random_state=10)

In [19]:
Xtrain.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
57,3,male,28.5,0,0,7.2292,C
717,2,female,27.0,0,0,10.5000,S
431,3,female,NaN,1,0,16.1000,S
633,1,male,NaN,0,0,0.0000,S
163,3,male,17.0,0,0,8.6625,S


In [20]:
Xval.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
590,3,male,35.0,0,0,7.1250,S
131,3,male,20.0,0,0,7.0500,S
628,3,male,26.0,0,0,7.8958,S
195,1,female,58.0,0,0,146.5208,C
230,1,female,35.0,1,0,83.4750,S


## Pré - processamento

In [71]:
pipe_categorica = Pipeline([
    
    ('imp', SimpleImputer(strategy='most_frequent' , fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  
])

In [72]:
pipe_numerica = Pipeline([
    
    
    ('imp',SimpleImputer(strategy='median')),
    ('scaler',MinMaxScaler())    
])

In [73]:
pipeline = ColumnTransformer([
    
    ('cat',pipe_categorica,cat_features),
    ('num',pipe_numerica,numeric_features)
    
])

## Modelo

In [74]:
seed = 10

In [75]:
rf_model = RandomForestClassifier(n_estimators=200,max_depth=6,random_state=seed)
lr_model = LogisticRegression(random_state = seed)

In [76]:
pipeline_random_fores = make_pipeline(pipeline,rf_model)

In [77]:
pipeline_random_fores.fit(Xtrain,ytrain)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imp',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked']),
                                                 ('num',
                                                  Pipeline(steps=[('imp',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare'])])),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=6, n_estimators=200,
                                        random_state=10))])

In [87]:
pipeline_random_fores.score(Xval,yval)

0.8379888268156425

In [88]:
pipeline_log_reg = make_pipeline(pipeline,lr_model)

In [89]:
pipeline_log_reg.fit(Xtrain,ytrain)
pipeline_log_reg.score(Xval,yval)

0.8100558659217877

### Usando o Cross validations

In [90]:
kfold = KFold(n_splits=10,shuffle=True,random_state=seed)

In [93]:
crosval  = cross_val_score (pipeline_random_fores,X,y,cv=kfold)
crosval.mean()

0.8260049937578027

In [95]:
def intervalo_recall(results):
    mean = results.mean()
    dv = results.std()
    print('Recall médio: {:.2f}%'.format(mean*100))
    print('Intervalo de Recall: [{:.2f}% ~ {:.2f}%]'
          .format((mean - 2*dv)*100, (mean + 2*dv)*100))



In [96]:
intervalo_recall(crosval)

Recall médio: 82.60%
Intervalo de Recall: [73.36% ~ 91.85%]
